In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException ,ElementNotVisibleException, ElementNotInteractableException,TimeoutException, ElementClickInterceptedException
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from datetime import datetime
import pandas as pd
from hdfs import InsecureClient



In [2]:
print("a")

a


In [3]:
def get_country():
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options = chrome_options)
    try:
        url = "https://www.flightradar24.com/data/airports"
        driver.get(url)
        # Find all links to countries
        country_links = driver.find_elements(By.CSS_SELECTOR, 'table#tbl-datatable a[href^="https://www.flightradar24.com/data/airports/"]')
        country = []
        # Extract and print the country names
        for country_link in country_links:
            country_name = country_link.get_attribute("title")
            country.append(country_name.replace(" ", "-").replace("(", "").replace(")", ""))
        country_set = set(country)
    finally:
        driver.quit()
        return (country_set)

In [4]:

def scrape_airport_data(country_url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options = chrome_options)

    try:
        driver.get(country_url)

        # Find all airport links
        IATA = []
        airport_names = []
        #k = 0
        airport_links = driver.find_elements(By.CSS_SELECTOR, 'a[data-iata][data-lat][data-lon]')
        for airport_link in airport_links:
            airport_name = airport_link.text.strip().split('\n')[0]
            airport_iata = airport_link.get_attribute('data-iata')
            airport_icao = airport_name.split('(')[-1].split(')')[0]
            IATA.append(airport_iata)
            airport_names.append(airport_name)
        airport_names_cleaned = [airport.split(' (')[0] for airport in airport_names]
 
    finally:
        driver.quit()
    return IATA , airport_names_cleaned


In [5]:
def scrape_all_countries_airports(main_url):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options = chrome_options)

    try:
        driver.get(main_url)

        # Find all links to countries
        country_links = driver.find_elements(By.CSS_SELECTOR, 'table#tbl-datatable a[href^="https://www.flightradar24.com/data/airports/"]')
        #p = 0
        AEROPORT_IATA = {}
        for country_link in set(country_links):
            country_url = country_link.get_attribute("href")
            print(country_url)
            for elem in scrape_airport_data(country_url)[0] : 
                AEROPORT_IATA[elem] = country_url[44:]

            #p = p+1
           # if p>10 :
               # break 
    finally:
        driver.quit()
    return AEROPORT_IATA


In [6]:
def click_button( driver , button_text):
            k = 0
            while (True and k<3):
                k = k+1
                try:
                    # Wait for the button to be present and visible
                    button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, f'//button[text()="{button_text}"]')))
                    # Click the button
                    button.click()
                    time.sleep(3)
                except (ElementClickInterceptedException, ElementNotInteractableException, TimeoutException) as e:
                    break
                except (ElementNotVisibleException, TimeoutException):
                    # Break the loop if the button is not visible or timeout occurs
                    break 

In [7]:
def scrape_flight_data (airport_code) :
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options = chrome_options)
    airport_url = "https://www.flightradar24.com/data/airports/"+airport_code+"/reviews"
    df = "no review data"  # Initialize df outside the try block
    try:
        driver.get(airport_url)

        # Wait for the button to appear and accept cookies
        time.sleep(12)
        try:
            button1 = driver.find_element(By.ID, 'onetrust-accept-btn-handler')
            button1.click()
        except NoSuchElementException:
            pass  # If the accept cookies button is not found, continue without clicking
        
        """def click_button(ButtonName):
            k = 0
            while k < 10:
                try:
                    # Attente explicite pour l'élément
                    button = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, 'btn-load-reviews')))
                    button.click()
                    k += 1
                except (ElementClickInterceptedException, ElementNotInteractableException):
                    break
                except TimeoutException:
                    # Si le bouton n'est pas trouvé dans le délai, sortir de la boucle
                    break"""
        try : 
            no_comment = driver.find_element(By.CLASS_NAME , 'default-tag')
        except : 
            no_comment = None

        if  no_comment is not None and no_comment.text == "No comments available for this airport"  : 
            return "no review data"
        else :
            click_button(driver , "Load more reviews")
            page_source = driver.page_source
            # Close the WebDriver
            driver.quit()
            # Parse the HTML code
            soup = BeautifulSoup(page_source, 'html.parser')
            #print(soup)
            elements = soup.find_all('div', class_='col-xs-10')
            filtered_elements = []
            for element in elements:
                stars_div = element.find('div', class_='stars')
                content_div = element.find('div', class_='content')
                subratings_div = element.find('div', class_='subratings')
                if stars_div and content_div and subratings_div:
                    # Si tous les éléments requis sont trouvés, ajoutez cet élément à la liste filtrée
                    filtered_elements.append(element)
                    # Extract the content of each element
                review_data = []
                for e in filtered_elements :   
                    stars_title = e.find(class_='stars')['title']
                    content = e.find(class_='content').text.strip()
                    subratings = e.find_all(class_='col-md-6')
                    dictionnaire = {}
                    date_element = e.find('span', class_='date pull-right')
                    if date_element:
                        date = date_element.get('title')
                    dictionnaire["date"]= date
                    dictionnaire["Stars Title"] = stars_title
                    dictionnaire ["text review"]= content
                    for subrating in subratings:
                        subrating_title = subrating['title']
                        label = subrating.find(class_='label').text.strip()
                        dictionnaire[label]= subrating_title
                    review_data.append(dictionnaire)
                df = pd.DataFrame(review_data)
        return df
                
    finally : 
        pass

In [ ]:
scrape_flight_data("alg")

In [8]:
def scrape_flight_from_country (country_name):
    url_country = "https://www.flightradar24.com/data/airports/" + country_name
    IATA , aeroport_names = scrape_airport_data(url_country)
    data_total_country = []
    for i, j in  zip (IATA, aeroport_names) : 
        df_aeroport = scrape_flight_data(i) 
        if isinstance(df_aeroport, pd.DataFrame):
            print("data founded in ",j , " = " , len(df_aeroport)  )
            df_aeroport["Destination Aeroport"] =  j 
            data_total_country.append(df_aeroport)
    if len(data_total_country) >0 :
        concatenated_df = pd.concat(data_total_country, ignore_index=True)
        return concatenated_df 


In [9]:
def send_to_hdfs(data, file_path): 
    client = InsecureClient('http://localhost:50070')
    with client.write(file_path, overwrite=True) as writer:
        data.to_csv(writer, index=False)


In [10]:
import os
import csv
from hdfs import InsecureClient

def save_csv(data, parent_folder, folder_name, file_name):
    # Create the folder if it doesn't exist
    folder_path = os.path.join(parent_folder, folder_name)
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # Define the file path within the new folder
    file_path = os.path.join(folder_path, file_name)

    send_to_hdfs(data , file_path.replace(os.path.sep, '/'))


In [11]:
from IPython.display import HTML
def scrape_all_reviews() :
    countries = get_country()
   # countries_done = ["India","Antarctica","Sri-Lanka","Dominica","Belize","Gibraltar","Sweden","North-Macedonia","Mexico",
                   ##   "Lithuania","Mozambique","Cook-Islands","Democratic-Republic-Of-The-Congo","Reunion","Singapore","Saint-Vincent-And-The-Grenadines","Mauritania","Zambia","Wallis-And-Futuna" ,"Cameroon","Belarus","Kiribati","South-Korea","Uganda","Norway","Guatemala","Poland","Martinique","Greece","Bhutan","Indonesia", "Jamaica","Gabon","Fiji","Oman","Niger",'Syria',"Tunisia",
                   ##   "Costa-Rica","Ghana","Algeria","Bosnia-And-Herzegovina"]
    for country in countries :
       # if country not in countries_done :
            print("start of extract data from :", country)
            data = scrape_flight_from_country(country) 
            print("End of extract data from :", country , "------------------------")
           # countries_done.append(country)
            
            date = datetime.now().date()
            # File path to save the CSV data
            file_name = country + str(date) +".csv"
            # Save the DataFrame to a CSV file
            parent_folder = "/user/PFE_data/reviews_flights"
            #folder name 
            folder_name = country

            if isinstance(data, pd.DataFrame) :
                save_csv(data,parent_folder ,folder_name,file_name )
                message = '<p style="color:green;">data saved</p>'
                display(HTML(message))

        

In [ ]:
scrape_all_reviews()

start of extract data from : Canada
data founded in  Abbotsford International Airport  =  1
